In [11]:
import os
from datetime import datetime

import yaml
from IPython.display import display, Markdown
from git import Repo as RepoLoader  # gitpython
from tqdm import tqdm

import papers_analyser.github as github
import papers_analyser.db as db
from papers_analyser.paper import load_papers,download_json_files

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
project_dir = os.path.abspath("..")

## Connect to DB

In [7]:
connection = db.create_connection( project_dir + "/data/paper.db")
db.create_table_if_not_exist(connection)

2.6.0


## Github API auth-token

In [3]:
auth_token = github.load_auth_token("config.yaml")

C:\Users\Peter-NB\Dev\reproduce-ml-papers\papers_analyser\github.py:15: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(open(path, "r"))


# Loading papers

In [4]:
json_path = project_dir + "/data"

Download JSON file if you have not done this already.

In [6]:
download_json_files(json_path)

Load json files and create paper objects that map to repositories.

In [5]:
since_date = datetime.strptime("2019-01-01", '%Y-%m-%d')


In [6]:
papers= load_papers(project_dir + "/data",  since_date)


In [15]:
papers_persisted = db.get_papers(connection)
papers_to_scrape = [paper for paper in papers if paper.url not in papers_persisted]

Scrape the file-names and the whole readme-file from the github-api.

In [ ]:
for paper in tqdm(papers_to_scrape):
    for repo in paper.repos:
        if not repo.scraped:
            repo.scrape(auth_token)

 75%|███████▌  | 7127/9449 [1:46:18<07:33,  5.12it/s]   

## Take a look on README files

In [22]:
readmes = [paper.repo[0].readme for paper in papers]

NameError: name 'papers' is not defined

In [ ]:
display(Markdown(readmes[4]))

# Save to database

In [15]:
for paper in papers_to_scrape:
    if paper.repos_scraped():
        db.insert_paper(connection,paper)

connection.commit()

In [ ]:
len(papers_to_scrape)

In [28]:
connection.close()

## Clone repo

In [36]:
dir = driverPath = project_dir + "/repos"
for paper in papers:
    paper_path = dir + "/" + paper.title.replace( ":","")
    if not os.path.exists(paper_path):
        os.mkdir(paper_path)
        
    for repo in paper.repo:
        repo_path = paper_path+ repo.repo_name.replace("/","_")
        
        if not os.path.exists(repo_path):
            os.mkdir(repo_path)
            
        RepoLoader.clone_from(repo.clone_url,repo_path)
    

